In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%html
<style>
.individual-selected {
    background-color: #90ee90;
}
</style>

In [ ]:
import ipywidgets as widg

In [ ]:
import pickle
from pathlib import Path

In [ ]:
from genetic_musical_generator.random_genome_to_midi import random_genome, genome2midi
from genetic_musical_generator.beat_fit import BeatFit, Offset, play, outport
from genetic_musical_generator.bass_fit import BassFit

In [ ]:
from dataclasses import dataclass
from typing import Optional

In [ ]:
@dataclass
class Individual:
    genome: str
    rating: Optional[float] = None
    beat: Optional[BeatFit] = None
    bass: Optional[BassFit] = None
    generation: int = 0

In [ ]:
savedir = Path('populations')
savedir.mkdir(exist_ok=True)
savepath = savedir/'impact_exhibit.pop'

In [ ]:
if savepath.exists():
    with savepath.open('rb') as f:
        population = pickle.load(f)
else:
    population = []

In [ ]:
from functools import partial
import mido
import threading

In [ ]:
def play_thread(midi, outport, kill, button):
    debug.value = str(midi)
    for m in midi.play():
        if kill.is_set():
            break
        outport.send(m)
    outport.reset()
    button.description = 'Hear'


class Player:
    def __init__(self):
        self.kill = None
        self.outport = mido.open_output('mido_out', virtual=True)
        
    def play(self, midi, button):
        self.stop()
        self.kill = threading.Event()
        self.thread = threading.Thread(target=play_thread, args=(midi, self.outport, self.kill, button))
        self.thread.start()
                
    def stop(self):
        if self.kill is not None:
            self.kill.set()
            self.thread.join()
            
    def __del__(self):
        self.outport.close()

In [ ]:
player = Player()

In [ ]:
def play_or_stop(button, midi):
    if button.description == 'Hear':
        button.description = 'Stop'
        player.play(midi, button)
    else:
        button.description = 'Hear'
        player.stop()

In [ ]:
from collections import deque

In [ ]:
def selected_refresh():
    for i, ind in enumerate(individuals):
        ind.remove_class('individual-selected')
        if i in selected:
            ind.add_class('individual-selected')
            
    breed_selected_button.disabled = False if len(selected) == 2 else True

In [ ]:
def select_individual(i):
    if i in selected:
        return
    selected.append(i)
    selected_refresh()
    debug.value = str(selected)

In [ ]:
from genetic_musical_generator.random_genome_to_midi import random_genome

In [ ]:
def generate_random(_):
    population.append(Individual(random_genome(40)))
    refresh_individual_list()

In [ ]:
from genetic_musical_generator.breed import mutate_crossover

In [ ]:
def breed_selected(_=None):
    assert len(selected) == 2
    population.append(Individual(mutate_crossover(*(population[s].genome for s in selected)), 
                                 generation=max([population[s].generation for s in selected])+1))
    refresh_individual_list()

In [ ]:
from random import choices

In [ ]:
def let_breed(_):
    draw = partial(choices, range(len(population)), 
                   weights=[0 if p.rating is None else p.rating for p in population])

    selected.extend(draw(k=2))
    while selected[0] == selected[1]:
        selected.extend(draw(k=1))
        
    selected_refresh()
    breed_selected()

In [ ]:
def save():
    with savepath.open('wb') as f:
        pickle.dump(population, f)

In [ ]:
def refresh_individual_list():
    global individuals, individual_list, breed_selected_button, generate_random_button, let_breed_button
    individuals = [controls(p,i) for i,p in enumerate(population)]
    individual_list = widg.VBox(individuals)
    
    generate_random_button = widg.Button(description='Generate Random')
    generate_random_button.on_click(generate_random)
    
    breed_selected_button = widg.Button(description='Breed Selected', disabled=True)
    breed_selected_button.on_click(breed_selected)
    
    let_breed_button = widg.Button(description='Let Breed', button_style='primary')
    let_breed_button.on_click(let_breed)
    
    save()
    
    output.clear_output()
    with output:
        display(widg.VBox([
            individual_list,
            widg.VBox([
                generate_random_button,
                breed_selected_button,
                let_breed_button
            ])
        ]))

    selected_refresh()

In [ ]:
def kill_individual(i):
    del population[i]
    delete = []
    for si,s in enumerate(selected):
        if s > i:
            selected[si] -= 1
        elif s == i:
            delete.append(si)
    for si in delete:
        del selected[si]
        
    debug.value = str(selected)

    refresh_individual_list()

In [ ]:
def assign_rating(i, change):
    population[i].rating = change['new']
    save()

In [ ]:
def controls(individual, i):
    hear_button = widg.Button(description='Hear', layout=widg.Layout(width='60px'))
    hear_button.on_click(lambda button: play_or_stop(button, genome2midi(individual.genome)))
    
    select_button = widg.Button(description='Select', layout=widg.Layout(width='60px'))
    select_button.on_click(lambda button: select_individual(i))
    
    kill_button = widg.Button(description='Kill', layout=widg.Layout(width='60px'))
    kill_button.on_click(lambda button: kill_individual(i))
    
    rating_box = widg.FloatText(description='Rating', value=individual.rating, 
                       layout=widg.Layout(flex='0 1 auto', width='auto'))
    rating_box.observe(partial(assign_rating, i), names='value')
    
    return widg.HBox([
        widg.Label(individual.genome, layout=widg.Layout(width='60%')), 
        hear_button,
        select_button,
        kill_button,
        rating_box
    ])

In [ ]:
selected = deque(maxlen=2)

In [ ]:
debug = widg.Label(layout={'height': '30px', 'display': 'hidden'})
debug

In [ ]:
output = widg.Output()
refresh_individual_list()
output

In [ ]:
space = widg.Label(layout={'height': '3000px'})
space